In [ ]:
!pip install -U --quiet transformers accelerate
!pip install rarfile
!apt-get install -y unrar
!pip install jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newe

## Loading input


In [ ]:
import json
from pandas import DataFrame
data = None
with open('predicted_data.json') as j:
  data = json.load(j)
print(len(data))
# data = data[:1]
DataFrame(data).head()

300


,user_input,retrieved_contexts,response,reference
0,- من هو جمال أحمد حمزة خاشقجي؟,[جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدين...,جمال أحمد حمزة خاشقجي هو صحفي إعلامي سعودي ولد...,صحفي وإعلامي
1,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,[جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدين...,جمال أحمد حمزة خاشقجي وُلِد في 13 أكتوبر 1958 ...,حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة -...
2,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,[جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدين...,مدينة الولادة لجمال أحمد حمزة خاشقجي هي المدين...,المدينة المنورة
3,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,[قبل دخوله السياسة، كان طبيبا، وتخصص في طب الع...,في واشنطن بوست.,واشنطن بوست
4,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,[كما أنها أيضًا عضو في منظمات متعددة الأطراف ب...,No_answer,"وُصف في الصحف وأجهزة الاعلام العالمية بأنه ""وف..."


## Evaluating

In [ ]:
import gdown
gdown.download(id="1RfCr1bhBLHlSJdteSs9Qbga2L7dpGiNQ", output='/content/metrics_prompts.zip')
!unzip -o "metrics_prompts.zip" -d "prompts"

Downloading...
From: https://drive.google.com/uc?id=1RfCr1bhBLHlSJdteSs9Qbga2L7dpGiNQ
To: /content/metrics_prompts.zip
100%|██████████| 18.6k/18.6k [00:00<00:00, 33.2MB/s]

Archive:  metrics_prompts.zip
   creating: prompts/flow_judge/
  inflating: prompts/flow_judge/context_precision.txt  
  inflating: prompts/flow_judge/context_recall.txt  
  inflating: prompts/flow_judge/factual_correctness.txt  
  inflating: prompts/flow_judge/faithfulness.txt  
  inflating: prompts/flow_judge/response_relevancy.txt  
  inflating: prompts/flow_judge/_metric description prompt.txt  
  inflating: prompts/flow_judge/_prompt generating prompt.txt  
   creating: prompts/m_promotheus/
  inflating: prompts/m_promotheus/context_precision.txt  
  inflating: prompts/m_promotheus/context_recall.txt  
  inflating: prompts/m_promotheus/factual_correctness.txt  
  inflating: prompts/m_promotheus/faithfulness.txt  
  inflating: prompts/m_promotheus/response_relevancy.txt  
  inflating: prompts/m_promotheus/_prompt generating prompt for MP.txt  


In [ ]:
import zipfile
from jinja2 import Template

def extract_prompt_templates(zip_path):
    """
    Extract Jinja templates from a zip file containing prompt templates.

    Args:
        zip_path (str): Path to the zip file

    Returns:
        dict: Nested dictionary {folder_name: {metric_name: jinja2.Template}}
    """
    prompt_templates = {}
    with zipfile.ZipFile(zip_path, 'r') as z:
        for file_info in z.infolist():
            # Skip directories and hidden files
            if file_info.is_dir():
                continue

            path_parts = file_info.filename.split('/')
            # Only process files in immediate subdirectories
            if len(path_parts) != 2:
                continue

            folder, filename = path_parts
            # Skip non-target folders and hidden files
            if folder not in ['flow_judge', 'm_promotheus']:
                continue
            if filename.startswith('_') or not filename.endswith('.txt'):
                continue

            # Extract metric name (remove .txt extension)
            metric_name = filename[:-4]

            # Read and decode file content
            with z.open(file_info) as f:
                content = f.read().decode('utf-8')

            # Create Jinja template
            template = Template(content)

            # Add to nested dictionary
            if folder not in prompt_templates:
                prompt_templates[folder] = {}
            prompt_templates[folder][metric_name] = template

    return prompt_templates


In [ ]:
metrics_prompts = extract_prompt_templates('./metrics_prompts.zip')
metrics_setup_args = {'flow_judge': ['inputs', 'response'],
                      'm_promotheus': ['data', 'response', 'reference']}
eval_config = {
    'metrics': ['faithfulness', 'context_precision', 'context_recall', 'response_relevancy', 'factual_correctness'],
    'default_metrics': ['factual_correctness'],
    'judge': 'flowaicom/Flow-Judge-v0.1', # 'flowaicom/Flow-Judge-v0.1' (good), 'Unbabel/M-Prometheus-3B' (very bad)
    'judge_name': 'flow_judge',
    'eval_prompts': metrics_prompts,
    'eval_setup_input': metrics_setup_args,
    'batch_size': 2,
    'logging': True
}


def log_for_me(message):
  if eval_config['logging']:
    print(message)

In [ ]:
print(metrics_prompts)
# print(metrics_prompts['flow_judge']['faithfulness'].render())

{'flow_judge': {'context_precision': <Template memory:7e5690a2de90>, 'context_recall': <Template memory:7e56976c8490>, 'factual_correctness': <Template memory:7e56906f41d0>, 'faithfulness': <Template memory:7e568f5b6950>, 'response_relevancy': <Template memory:7e5690347b10>}, 'm_promotheus': {'context_precision': <Template memory:7e568f5b6410>, 'context_recall': <Template memory:7e5690704f50>, 'factual_correctness': <Template memory:7e56906f44d0>, 'faithfulness': <Template memory:7e5690ef6d50>, 'response_relevancy': <Template memory:7e568f5b6cd0>}}


In [ ]:
#### loading the model
from transformers import pipeline
import torch

model_id = eval_config['judge']
pipe = pipeline(
          "text-generation",
          model = model_id,
          max_new_tokens=1024,
          device="cuda",
          return_full_text=False,
          torch_dtype = torch.bfloat16,
          batch_size = eval_config['batch_size'],
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
#### setup functions
def flow_judge_data_builder(data, with_context= False):
  ready_data = []
  for row in data:
    inputs, response = "", ""

    inputs = f"""
    <user_query>
    {row['user_input']}
    </user_query>

    <reference answer>
    {row['reference']}
    </reference answer>
    """
    if with_context:
      contexts= "\n".join(row['retrieved_contexts'])
      inputs += f"""<provided information>
      {contexts}
      </provided information>"""

    response = row['response']

    ready_data.append({'inputs':inputs, 'response':response})
  return ready_data

def m_promotheus_data_builder(data, with_context= False):
  ready_data = []
  for row in data:
    data__in, response, reference = "", "", ""

    data__in = f"""
    ## user_query:
    {row['user_input']}"""

    if with_context:
      contexts = "\n".join(row['retrieved_contexts'])
      inputs += f"""## provided information to answer user query:
      {contexts}"""

    reference = row['reference']
    response = row['response']

    ready_data.append({'data':data__in,'reference':reference, 'response':response})
  return ready_data

def extract_tag_from_structured_answer(answer, begin_tag, end_tag = None, converter = None):
  if end_tag:
    val = answer.split(end_tag)[0].split(begin_tag)[1].strip()
  else:
    val = answer.split(begin_tag)[1].strip()
  if converter:
    val = converter(val)
  return val

def flow_judge_feedback_extractor(answer):
  return extract_tag_from_structured_answer(answer,'<feedback>','</feedback>')

def m_promotheus_feedback_extractor(answer):
  return extract_tag_from_structured_answer(answer,'Feedback:','[RESULT]')

def flow_judge_score_extractor(answer):
  return extract_tag_from_structured_answer(answer,'<score>','</score>')

def m_promotheus_score_extractor(answer):
  return extract_tag_from_structured_answer(answer,'[RESULT]')

judge_handlers = { 'builder' : {'flow_judge': flow_judge_data_builder,
                                'm_promotheus': m_promotheus_data_builder
                                },
                   'feedback_extractor':{'flow_judge': flow_judge_feedback_extractor,
                                'm_promotheus': m_promotheus_feedback_extractor
                                },
                   'score_extractor': {'flow_judge': flow_judge_score_extractor,
                                'm_promotheus': m_promotheus_score_extractor
                                },
}

def setup_data(data, judge):
  print(f"setting inputs for judge {judge} which are {eval_config['eval_setup_input'][judge]}'")
  return judge_handlers['builder'][judge](data)

def process_tasks(tasks, prompt_to_messages = True):
  length_of_m_tasks = len(list(tasks.values())[0])
  flattened_tasks = sum([ts for m,ts in tasks.items()],[])

  from tqdm.notebook import tqdm

  # Generate the grouped processing list
  remaining = len(flattened_tasks)
  to_processing_list = []
  start_index = 0

  # First batch: size 1
  if remaining > 0:
      batch_size = 1
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Next batch: size 2
  if remaining > 0:
      batch_size = min(2, remaining)
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Next batch: size 4
  if remaining > 0:
      batch_size = min(4, remaining)
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Subsequent batches: size 8 until completion
  while remaining > 0:
      batch_size = min(8, remaining)
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Process tasks with dynamic batch size progress
  flattened_results = []
  converter_to_messages = lambda x : [ {"role": "user", "content": x}]
  with tqdm(total=len(flattened_tasks)) as pbar:
      for batch in to_processing_list:
          batch = [converter_to_messages(t) for t in batch] if prompt_to_messages else batch
          flattened_results += (pipe(batch))
          pbar.update(len(batch))

  # collect results
  results = []
  temp = []
  for res in flattened_results:
    temp.append(res)
    if len(temp) == length_of_m_tasks:
      results.append(temp)
      temp = []

  return results



def run_evaluation(data, metrics = None):
  data_as_params = setup_data(data=data, judge=eval_config['judge_name'])
  log_for_me('finished data setup')

  using_metrics = metrics or eval_config['default_metrics']
  prompts = [(m, eval_config['eval_prompts'][eval_config['judge_name']][m]) for m in using_metrics]
  prompts = dict(prompts)

  tasks = {m: [prompt.render(params) for params in data_as_params] for m, prompt in prompts.items()}
  log_for_me('finished preparing tasks')

  results = process_tasks(tasks)

  tasks_results = dict(zip(using_metrics, results))
  # extracting results
  feedback_ext = judge_handlers['feedback_extractor'][eval_config['judge_name']]
  score_ext = judge_handlers['score_extractor'][eval_config['judge_name']]
  detailed = {}
  for m, res in tasks_results.items():
    new_keys = m+'_feedback', m+'_score'
    temp = {new_keys[0]:[], new_keys[1]:[]}
    for r in res:
      answer = r[0]['generated_text']
      new_vals = feedback_ext(answer), score_ext(answer)
      temp[new_keys[0]].append(new_vals[0])
      temp[new_keys[1]].append(new_vals[1])
    detailed.update(temp)
  for m in using_metrics: # extraction is done and no need for the generated text
      tasks_results.pop(m)
  tasks_results.update(detailed)
  # preparing final output
  from pandas import DataFrame, concat
  df1,df2 = DataFrame(tasks_results), DataFrame(data)
  df = concat([df1,df2], axis = 1)
  complete_results = df.to_dict(orient = 'records')
  return complete_results


In [ ]:
## running evaluation
results = run_evaluation(data)

setting inputs for judge flow_judge which are ['inputs', 'response']'
finished data setup
finished preparing tasks


  0%|          | 0/300 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


## saving results

In [ ]:
from pandas import DataFrame
df = DataFrame(results)
df.to_json('evaluation_results__by_' + eval_config['judge_name'] + '.json', orient='records', indent=4)
print(df['retrieved_contexts'].iloc[0])
df.head()

['جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا.جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا. ويكتب عموداً في صحيفة واشنطن بوست منذ 2017، وُصف في الصحف وأجهزة الاعلام العالمية بأنه "وفيّ للدولة السعودية" و"منتقد لسياساتها".غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ذلك مقالات صحفية انتقد فيها الحكومة السعودية. انتقد خاشقجي بصورة كبيرة ولي العهد السعودي محمد بن سلمان، والملك سلمان بن عبد العزيز. ', 'انتقد خاشقجي بصورة كبيرة ولي العهد السعودي محمد بن سلمان، والملك سلمان بن عبد العزيز. وكذلك عارض التدخل العسكري في اليمن.', 'جمال عبد الناصر (15 يناير 1918 – 28 سبتمبر 1970). هو ثاني رؤساء مصر. تولى السلطة من سنة 1956 إلى وفاته. ']


,context_recall_feedback,context_recall_score,user_input,retrieved_contexts,response,reference
0,The output provides a detailed biography of Ja...,4,- من هو جمال أحمد حمزة خاشقجي؟,[جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدين...,جمال أحمد حمزة خاشقجي هو صحفي إعلامي سعودي ولد...,صحفي وإعلامي
1,The output provided by the AI system fully sup...,5,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,[جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدين...,جمال أحمد حمزة خاشقجي وُلِد في 13 أكتوبر 1958 ...,حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة -...
2,The output provided by the AI system perfectly...,5,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,[جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدين...,مدينة الولادة لجمال أحمد حمزة خاشقجي هي المدين...,المدينة المنورة
3,The task requires evaluating how well the retr...,4,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,[قبل دخوله السياسة، كان طبيبا، وتخصص في طب الع...,في واشنطن بوست.,واشنطن بوست
4,The output provided does not contain any infor...,1,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,[كما أنها أيضًا عضو في منظمات متعددة الأطراف ب...,No_answer,"وُصف في الصحف وأجهزة الاعلام العالمية بأنه ""وف..."


## Fallback Evaluations when model is not enough
using Bleu and Rouge for answer and some embedding eval for retrieval.

In [ ]:
!pip install --quiet ragas sacrebleu rouge_score

In [ ]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore
from ragas.metrics import RougeScore

from tqdm.notebook import tqdm

def traditional_evaluation(data):
  # create samples

  samples = [SingleTurnSample(response=item['response'], reference=item['reference']) for item in data]

  bleu_scorer = BleuScore()
  rouge1_scorer = RougeScore(rouge_type="rouge1")
  rougeL_scorer = RougeScore(rouge_type="rougeL")
  scores = []
  for sample in tqdm(samples):
    score = {'bleu': bleu_scorer.single_turn_score(sample),
             'rouge1': rouge1_scorer.single_turn_score(sample),
             'rougeL': rougeL_scorer.single_turn_score(sample)}
    scores.append(score)

  return scores

In [ ]:
trad_results = traditional_evaluation(data)

In [ ]:
# combine with other results
from pandas import DataFrame, concat
df1, df2 = DataFrame(results), DataFrame(trad_results)
df = concat([df1,df2], axis=1)
df.to_json('evaluation_results__by_' + eval_config['judge_name'] + '.json', orient='records', indent=4)
df.head()